<a href="https://colab.research.google.com/github/truboxl/boinc-colab/blob/master/boincappdata-colab-stable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BOINC for Google Colab (CPU)

This is a Colab notebook for running BOINC on Google Colab.
Written by [truboxl](https://github.com/truboxl).

Licensed under MIT License.

See Table of contents at the left sidebar for different sections.

In [ ]:
# add code line 16-48 to F12 console to prevent disconnect
# or use Tampermonkey
'''
// ==UserScript==
// @name         Colab Auto Reconnect Script
// @namespace    https://github.com/truboxl
// @version      2021-01-10
// @description  Use CARS to stay connected with Google Colab
// @author       truboxl
// @match        *://colab.research.google.com/drive/*
// @grant        none
// ==/UserScript==

(function() {
    'use strict';
    // https://greasyfork.org/en/scripts/412404-colab-保持活跃-make-colab-keep-alive
    // 60010-80490 (1min+) is too short
    function timer() {
        var min = 5*60*1000
        var max = 8*60*1000
        var randomTime = parseInt(Math.random()*(max-min)+min, 10); // 5min+ x 60s x 1000ms
        console.log("Random time (ms) =", randomTime);
        console.log("Random time (min) =", randomTime/1000/60);
        return randomTime;
    }

    // https://www.quora.com/In-JavaScript-how-can-I-simulate-a-keystroke
    var ESCkey = new KeyboardEvent('keydown', {'keyCode':27, 'which':27});
    function FakeESC() {
        console.log("Faking ESC key");
        document.dispatchEvent(ESCkey);
    }

    // https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0
    function FakeClick() {
        console.log("Faking click");
        // colab-connect-button or colab-toolbar-button
        document.querySelector("colab-connect-button").click();
    }
    function FakeEvent() {
        FakeClick();
        console.log("Waiting 5min for next faking");
        console.log("Faking ESC key in 5s");
        setTimeout(FakeESC, 5000);
        setTimeout(FakeEvent, timer());
    }
    // main loop
    setTimeout(FakeEvent, timer());
})();
'''
# if you are OK, then "Run all"
!uptime && gcc -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -E -- '-march=|-mtune=' | cut -f3

## Connect Google Colab to Google Drive for persistent storage

A Google account is required to allowing storing BOINC data on Google Drive.

ALWAYS MAKE SURE NO ERROR OCCURED HERE!!! If so, do "Factory reset runtime"!



In [ ]:
%%time
%cd /content/
# https://stackoverflow.com/questions/55918562/changing-the-system-time-in-google-colaboratory
!rm /etc/localtime && ln -s /usr/share/zoneinfo/Asia/Kuala_Lumpur /etc/localtime && date && uptime

# allow access to Google Drive for storage, shorten entry to /content/gdrive/
from google.colab import drive
drive.mount('googledrive')
!uptime && ln -s 'googledrive/My Drive' gdrive

# you may want to clear your Trash in Google Drive to fix problems
# https://drive.google.com/drive/trash
# https://one.google.com/storage/management?from=1

In [ ]:
%%time
'''
There could be messages like below showing up, so beware!
"Colab is experiencing issues connecting to Drive, and we are actively investigating."
"Timeout Error"

Avoid projects that have deep folder structure to prevent Drive timeout error:
Rosetta@home
WCG ARP
'''
# set up BOINC data directory and permissions
!mkdir -p /content/gdrive/boincappdata/
%cd /content/gdrive/boincappdata/
!chmod u+rwx -R projects slots 2>>/content/error.txt
#!ls /content/gdrive/boincappdata/
!ls -la projects/*/* slots/*/* | grep 'rw-'
#!mount
#!whoami

## Update VM components and install build tools

In [ ]:
%%time
%cd /content/gdrive/boincappdata/
!chmod u+rwx -R projects slots 2>>/content/error.txt

!apt-get update 2>/dev/null >/dev/null
#!apt-get upgrade -y 2>/dev/null >/dev/null # can take some time, do we need to update all components? cuda is not updated for some reason
!add-apt-repository -y ppa:ubuntu-toolchain-r/test 2>/dev/null >/dev/null # gcc 7.5 and clang 6.0 are too old to support skylake+ hardware
!apt-get install -y gcc-10 g++-10 git make m4 automake autoconf libtool psmisc libjpeg62 2>/dev/null >/dev/null
#!apt-get install -y clang-10 2>/dev/null >/dev/null

## Wheel of fortune

In [ ]:
%%time
'''
You can try reroll your chance using "Factory Reset Runtime" but it doesn't always work
Probability of chip given by Google: (performance not listed in order)
znver2 (wrongly identified znver1):        ++
skylake-avx512 (wrongly identified knl):   ++
broadwell:                                 +++++++++
haswell:                                   ++++++++++
'''
!lscpu | grep -E 'CPU family:|Model:|Model name:' # sometimes Intel Xeon or AMD EPYC
!gcc-10 -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -E -- '-march=|-mtune=' | cut -f3 # Xeon has many generations, give me details
!gcc-10 -march=native -Q --help=target | sed 's|.*Known.*||g' | grep -- '-march=' | cut -f3 >/content/march.txt
#!echo | clang-10 -E - -march=native -\#\#\# # clang is a bit more ugly

# Compile BOINC from source

In [ ]:
%%time
# build BOINC from source because release version still dont have ability to change localhost name
%cd /content/
!git clone git://github.com/boinc/boinc

In [ ]:
%%time
%cd /content/boinc/
!sed -e 's/^if !OS_WIN32/if OS_DARWIN/' -i client/Makefile.am # disable building switcher
!grep 'if !OS_WIN32' client/Makefile.am

In [ ]:
%%time
%cd /content/boinc/
!gcc-10 -v 2>&1 | grep 'gcc version'
#!clang-10 -v 2>&1 | grep 'clang version'
import os
commonFLAGS = '-pipe -O2 -march=native -fstack-protector-strong -Werror=return-type -Werror=int-to-pointer-cast '
os.environ['CFLAGS']    = commonFLAGS + \
                          '-Werror=pointer-to-int-cast '+ \
                          '-Werror=implicit-function-declaration'
os.environ['CXXFLAGS']  = commonFLAGS
os.environ['CC']        = 'gcc-10'           # gcc-10 or clang-10
os.environ['CXX']       = 'g++-10'           # g++-10 or clang++-10
os.environ['AR']        = 'gcc-ar-10'        # gcc-ar-10 or llvm-ar-10
os.environ['RANLIB']    = 'gcc-ranlib-10'    # gcc-ranlib-10 or llvm-ranlib-10
os.environ['NM']        = 'gcc-nm-10'        # gcc-nm-10 or llvm-nm-10
!make -s distclean 2>/dev/null >/dev/null
!./_autosetup 2>/dev/null >/dev/null
!./configure --disable-server --disable-manager 2>/dev/null >/dev/null

In [ ]:
%%time
%cd /content/boinc/
!time make -s -j$(($(nproc)*3/2)) 2>/dev/null >/dev/null

# Install / restart BOINC

In [ ]:
%%time
%cd /content/boinc/
!while true; do \
    echo "Checking whether BOINC is running... $(pidof boinc)"; \
    if [ "$(pidof -s boinc)" = "" ]; then \
        break; \
    fi; \
    echo "Killing BOINC..."; \
    kill "$(pidof -s boinc)"; \
    sleep 10; \
done
!make -s install 2>/dev/null >/dev/null
!which boinc
!du /usr/local/bin/boinc*
!strip /usr/local/bin/boinc*
!du /usr/local/bin/boinc*

# Running BOINC

Remove previous logs and run benchmark first so that:

1. credits properly granted
1. compiled BOINC client does not crash

In [ ]:
%cd /content/gdrive/boincappdata
#%pycat /content/gdrive/boincappdata/cc_config.xml
# update device name before launching BOINC rather than after to avoid regen ID
!while true; do \
    echo "Checking whether BOINC is running... $(pidof boinc)"; \
    if [ "$(pidof -s boinc)" = "" ]; then \
        break; \
    fi; \
    echo "Killing BOINC..."; \
    kill "$(pidof -s boinc)"; \
    sleep 10; \
done
!grep '<device_name>' cc_config.xml
!sed -e "s|<device_name>.*|<device_name>Colab $(paste /content/march.txt) $(date +%Y%m%d)</device_name>|" -i cc_config.xml
!grep '<device_name>' cc_config.xml

In [ ]:
%%time
%cd /content/gdrive/boincappdata/
# if need run benchmark, append boinc with parameter --run_cpu_benchmarks
!if [ -z "$(pidof -s boinc)" ] && [ -z "$(paste /content/error.txt)" ]; then \
    rm -f time_stats_log stdoutdae.txt stderrdae.txt stdoutgpudetect.txt stderrgpudetect.txt; \
    echo 'Starting BOINC...'; \
    boinc --daemon --dir /content/gdrive/boincappdata/; \
    sleep 30; \
fi

# Managing BOINC

## Output

Running the first 2 should give you the overall view of VM usage by BOINC.

In [ ]:
%%time
!uptime
!pstree -al # check currently running processes
#!paste /proc/meminfo # check meminfo

In [ ]:
%%time
%cd /content/gdrive/boincappdata/
!uptime
#!paste stdoutdae.txt # show full log
#!paste stderrdae.txt
#!tail stdoutdae.txt # show last few lines
#!tail -f stdoutdae.txt # follow log
!tail -n -50 stdoutdae.txt # last 50 lines of log

## Control panel (boinccmd)

Use this to monitor BOINC client, set up projects, connect account managers

In [ ]:
%%time
%cd /content/gdrive/boincappdata/
!uptime && du -sh
#!boinc --help
#!boinccmd --help
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --lookup_account URL email passwd
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --project_attach URL auth
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --acct_mgr attach URL name passwd
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --network_available
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --acct_mgr sync
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --acct_mgr info
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --acct_mgr detach
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --project URL nomorework # reset | detach | update | suspend | resume | nomorework | allowmorework | detach_when_done | dont_detach_when_done
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --task URL task_name abort # suspend | resume | abort
#!time boinccmd --passwd $(paste gui_rpc_auth.cfg) --get_state #| grep 'request more work'
!time boinccmd --passwd $(paste gui_rpc_auth.cfg) --get_tasks | grep -E '   name|active_task_state|fraction done'

## Control panel (cc_config.xml)

Create this config file once

In [ ]:
#%cd /content/gdrive/boincappdata/
#%pycat /content/gdrive/boincappdata/cc_config.xml
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --read_cc_config

In [ ]:
'''
%%writefile /content/gdrive/boincappdata/cc_config.xml
<cc_config>
<options>
<device_name>Colab</device_name>
<no_alt_platform>1</no_alt_platform>
<report_results_immediately>1</report_results_immediately>
</options>
</cc_config>
'''

## Control panel (global_prefs_override.xml)

On demand compute, don't buffer tasks

In [ ]:
#%cd /content/gdrive/boincappdata/
#%pycat /content/gdrive/boincappdata/global_prefs_override.xml
#!boinccmd --passwd $(paste gui_rpc_auth.cfg) --read_global_prefs_override

In [ ]:
'''
%%writefile /content/gdrive/boincappdata/global_prefs_override.xml
<global_preferences>
   <run_on_batteries>1</run_on_batteries>
   <run_if_user_active>1</run_if_user_active>
   <run_gpu_if_user_active>1</run_gpu_if_user_active>
   <suspend_cpu_usage>0.000000</suspend_cpu_usage>
   <start_hour>0.000000</start_hour>
   <end_hour>0.000000</end_hour>
   <net_start_hour>0.000000</net_start_hour>
   <net_end_hour>0.000000</net_end_hour>
   <leave_apps_in_memory>0</leave_apps_in_memory>
   <confirm_before_connecting>0</confirm_before_connecting>
   <hangup_if_dialed>0</hangup_if_dialed>
   <dont_verify_images>0</dont_verify_images>
   <work_buf_min_days>0.000000</work_buf_min_days>
   <work_buf_additional_days>0.000000</work_buf_additional_days>
   <max_ncpus_pct>100.000000</max_ncpus_pct>
   <cpu_scheduling_period_minutes>60.000000</cpu_scheduling_period_minutes>
   <disk_interval>60.000000</disk_interval>
   <disk_max_used_gb>0.000000</disk_max_used_gb>
   <disk_max_used_pct>100.000000</disk_max_used_pct>
   <disk_min_free_gb>0.000000</disk_min_free_gb>
   <vm_max_used_pct>1.000000</vm_max_used_pct>
   <ram_max_used_busy_pct>75.000000</ram_max_used_busy_pct>
   <ram_max_used_idle_pct>90.000000</ram_max_used_idle_pct>
   <max_bytes_sec_up>0.000000</max_bytes_sec_up>
   <max_bytes_sec_down>0.000000</max_bytes_sec_down>
   <cpu_usage_limit>100.000000</cpu_usage_limit>
   <daily_xfer_limit_mb>0.000000</daily_xfer_limit_mb>
   <daily_xfer_period_days>0</daily_xfer_period_days>
</global_preferences>
'''

In [ ]:
#%cd /content/gdrive/boincappdata/
#%pycat /content/gdrive/boincappdata/client_state.xml

In [ ]:
#%%writefile /content/gdrive/boincappdata/client_state.xml